<b>Facility Location: Variant B</b>

<img src="facility location B.jpg" width=55% align="left">

### Facility Location Problem (Variant B):

***Let us denote:***

- $D$ as the set of all districts (from District 1 to District 16 in this case).
- $N_i$ as the set of neighboring districts of district $i$.
- $c_i$ as the cost for placing a fire station in district $i$, for $i \in D$.
- $x_i$ as the binary decision variable which equals 1 if a fire station is placed in district $i$, 0 otherwise, for $i \in D$.
- $y_{ij}$ as the binary decision variable which equals 1 if the fire station in district $i$ covers district $j$, 0 otherwise, for $i \in D$, $j \in N_i$.

**Objective Function:**

Minimize the total cost of placing the fire stations:

$$\min \sum_{i \in D} c_{i} \cdot x_{i}$$

**Constraints:**

1. `Coverage Constraint:` Each district must be covered by at least one fire station.

$$\sum_{i \in N_{j}} y_{ij} \ge 1, \quad \forall j \in D$$

2. `Capacity Constraint:` A fire station in a district can cover at most 3 districts.

$$\sum_{j \in N_{i}} y_{ij} \le 3 \cdot x_{i}, \quad \forall i \in D$$

3. `Binary Decision Variables:`

$$x_{i} \in \{0,1\}, \quad \forall i \in D$$

$$y_{ij} \in \{0,1\}, \quad \forall i \in D, j \in N_i$$

This problem aims to minimize the total cost of placing the fire stations while ensuring that each district is covered by at least one fire station and a fire station in a district does not cover more than 3 districts.

In [1]:
import csv
f = open("set_covering.csv")
csvfile = csv.DictReader(f, delimiter='\t')

headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

In [2]:
Districts = headers[:]
Districts.remove('District')

In [3]:
Cost = {}
for i in Districts:
    Cost[i] = float(table[0][i])

In [4]:
# create a set of neighbors for each district
Neighbors = {}
# loop over the rows of the adjacency matrix
for i in range(1,len(table)):
    # define empty set
    tmpSet = set()
    # loop over all districts (headers)
    for j in Districts:
        # if table == 1 : add the district to the set
        if table[i][j] == '1':
            tmpSet.add(j)
    # assign the set to the district corresponding to row i
    D = table[i]['District']
    Neighbors[D] = tmpSet

In [5]:
from docplex.mp.model import Model
mdl = Model()

In [6]:
# variables
select = mdl.binary_var_dict(Districts, name='select')

canCoverSet = { (i,j) for i in Districts for j in Neighbors[i] }
canCover = mdl.binary_var_dict(canCoverSet, name='can cover')

In [7]:
# objective
mdl.minimize(mdl.sum(Cost[i]*select[i] for i in Districts))

In [8]:
# constraints: cover each district
CoverDistrict = {}
for j in Districts:
    CoverDistrict[j] = mdl.add_constraint(mdl.sum(canCover[i,j] for i in Neighbors[j]) >= 1)

In [9]:
# capacity/linking constraints
for i in Districts:
    mdl.add_constraint(mdl.sum(canCover[i,j] for j in Districts if i in Neighbors[j]) <= 3*select[i])

In [10]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.266,status='integer optimal solution')

In [11]:
mdl.print_solution()

objective: 3310.000
  "select_D3"=1
  "select_D5"=1
  "select_D10"=1
  "select_D13"=1
  "select_D14"=1
  "select_D16"=1
  "can cover_D5_D8"=1
  "can cover_D5_D5"=1
  "can cover_D5_D1"=1
  "can cover_D16_D16"=1
  "can cover_D16_D13"=1
  "can cover_D14_D14"=1
  "can cover_D3_D2"=1
  "can cover_D10_D10"=1
  "can cover_D10_D4"=1
  "can cover_D16_D15"=1
  "can cover_D13_D7"=1
  "can cover_D13_D9"=1
  "can cover_D3_D6"=1
  "can cover_D13_D12"=1
  "can cover_D10_D11"=1
  "can cover_D3_D3"=1


In [12]:
# inspect how often each district is covered
for j in Districts:
    print("District " + j + ": " + str(CoverDistrict[j].lhs.solution_value))

District D1: 1
District D2: 1
District D3: 1
District D4: 1
District D5: 1
District D6: 1
District D7: 1
District D8: 1
District D9: 1
District D10: 1
District D11: 1
District D12: 1
District D13: 1
District D14: 1
District D15: 1
District D16: 1
